In [1]:
import os
import datetime
import errno

import numpy as np

import deepcell
from deepcell.utils.tracking_utils import load_trks, trks_stats

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [2]:
# The path to the data file is currently required for `train_model_()` functions

# Change DATA_DIR if you are not using `deepcell.datasets`

# DATA_FILE should be a trks file (contains 2 np arrays and a lineage dictionary)
##DATA_FILE = "../trks_141923_repeat.trks"
DATA_FILE = "../train.npz"
TEST_FILE = '../test.npz'  ## Added

DATA_DIR = '~'
# confirm the data file is available
assert os.path.isfile(DATA_FILE)
#assert os.path.isfile(TEST_FILE)

In [8]:
trks_stats('../train.trks')
trks_stats('../test.trks')

Dataset Statistics: 
Image data shape:  (440, 100, 530, 530, 1)
Number of lineages (should equal batch size):  440
Total number of unique tracks (cells)      -  988
Total number of divisions                  -  0
Average cell density (cells/100 sq pixels) -  0.0004974271012006861
Average number of frames per track         -  75
Dataset Statistics: 
Image data shape:  (66, 100, 530, 530, 1)
Number of lineages (should equal batch size):  66
Total number of unique tracks (cells)      -  174
Total number of divisions                  -  0
Average cell density (cells/100 sq pixels) -  0.0004903071296805722
Average number of frames per track         -  64


In [3]:
# Set up other required filepaths

# If the data file is in a subdirectory, mirror it in MODEL_DIR and LOG_DIR
PREFIX = 'test_0818'

ROOT_DIR = '../train_0818'  # TODO: Change this! Usually a mounted volume
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

In [4]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

n_epoch = 10    # Number of training epochs
test_size = .1  # % of data saved as validation
train_seed = 1   # Random seed for training/validation data split

optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
lr_sched = rate_scheduler(lr=0.01, decay=0.99)

# Tracking training settings
features = {'appearance', 'distance', 'neighborhood', 'regionprop'}
min_track_length = 9
neighborhood_scale_size = 1  ## 30
batch_size = 16  # changed  ## 128  
crop_dim = 40  # changed  ## 32
in_shape = (crop_dim, crop_dim, 1)

model_name = 'tracking_model_seed{}_tl{}'.format(train_seed, min_track_length)

In [5]:
import deepcell.image_generators as generators
from deepcell.utils.data_utils import get_data

# Get the data
#train_dict, test_dict = get_data(DATA_FILE, mode='siamese_daughters',
#                                 seed=train_seed, test_size=test_size)
train_dict, test_dict = get_data(DATA_FILE, mode='sample', seed=train_seed, test_size=test_size)

In [6]:
print(train_dict['X'].shape)
print(test_dict['X'].shape)

(396, 100, 530, 530, 1)
(44, 100, 530, 530, 1)


In [7]:
# Build the generators and iterators
datagen_train = generators.SiameseDataGenerator(
    rotation_range=0, # randomly rotate images by 0 to rotation_range degrees
    shear_range=0,      # randomly shear images in the range (radians , -shear_range to shear_range)
    horizontal_flip=0,  # randomly flip images
    vertical_flip=0)  # randomly flip images
 
train_data = datagen_train.flow(
    #test_dict,  # original
    train_dict,
    batch_size=batch_size,
    seed=train_seed,
    crop_dim=crop_dim,
    neighborhood_scale_size=neighborhood_scale_size,
    min_track_length=min_track_length,
    features=features,
    save_to_dir=ROOT_DIR)

In [8]:
datagen_test = generators.SiameseDataGenerator(
    rotation_range=0,  # randomly rotate images by 0 to rotation_range degrees
    shear_range=0,     # randomly shear images in the range (radians , -shear_range to shear_range)
    horizontal_flip=0, # randomly flip images
    vertical_flip=0)   # randomly flip images

test_data = datagen_test.flow(
    test_dict,
    batch_size=batch_size,
    seed=train_seed,
    crop_dim=crop_dim,
    neighborhood_scale_size=neighborhood_scale_size,
    min_track_length=min_track_length,
    features=features)

In [9]:
from deepcell import model_zoo

tracking_model = model_zoo.siamese_model(
    input_shape=in_shape,
    neighborhood_scale_size=neighborhood_scale_size,
    features=features)

In [10]:
from deepcell import losses

n_classes = tracking_model.layers[-1].output_shape[-1]

def loss_function(y_true, y_pred):
    return losses.weighted_categorical_crossentropy(y_true, y_pred,
                                                    n_classes=n_classes,
                                                    from_logits=False)

In [11]:
tracking_model.compile(loss=loss_function, optimizer=optimizer, metrics=['accuracy'])

### Train the model

Call `fit_generator` on the compiled model, along with a default set of callbacks.

In [12]:
steps_per_epoch= train_dict['y'].shape[0] // batch_size
b = np.array(steps_per_epoch).astype(np.float32)
print(b.dtype)

float32


In [13]:
from deepcell.utils.train_utils import get_callbacks
from deepcell.utils.train_utils import count_gpus
from deepcell.utils import tracking_utils


model_path = os.path.join(MODEL_DIR, '{}.h5'.format(model_name))
loss_path = os.path.join(MODEL_DIR, '{}.npz'.format(model_name))

num_gpus = count_gpus()

print('Training on', num_gpus, 'GPUs.')

train_callbacks = get_callbacks(
    model_path,
    lr_sched=lr_sched,
    tensorboard_log_dir=LOG_DIR,
    save_weights_only=num_gpus >= 2,
    monitor='val_loss',
    verbose=1)

# rough estimate for steps_per_epoch
#total_train_pairs = tracking_utils.count_pairs(train_dict['y'], same_probability=5.0)
#total_test_pairs = tracking_utils.count_pairs(test_dict['y'], same_probability=5.0)
steps_per_epoch= train_dict['y'].shape[0] // batch_size
validation_steps = test_dict['y'].shape[0] // batch_size

#print(steps_per_epoch.dtype)

Training on 0 GPUs.


In [14]:
# fit the model on the batches generated by datagen.flow()
loss_history = tracking_model.fit_generator(
    train_data,
    #steps_per_epoch=total_train_pairs // batch_size,
    steps_per_epoch=np.array(steps_per_epoch).astype(np.float32),
    validation_data=test_data,
    #validation_steps=total_test_pairs // batch_size,
    epochs=10,
    validation_steps = np.array(validation_steps).astype(np.float32),
    callbacks=train_callbacks)

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1935: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
24/24 [==============================] - 58s 2s/step - loss: 0.2356 - accuracy: 0.8802 - val_loss: 0.1942 - val_accuracy: 0.9688

Epoch 00001: val_loss improved from inf to 0.19417, saving model to /home/jupyter/train_0818/models/test_0818/tracking_model_seed1_tl9.h5
Epoch 2/10
24/24 [==============================] - 47s 2s/step - loss: 0.0943 - accuracy: 0.9505 - val_loss: 0.2535 - val_accuracy: 0.8438

Epoch 00002: val_loss did not improve from 0.19417
Epoch 3/10
24/24 [==============================] - 47s 2s/step - loss: 0.1026 - accuracy: 0.9505 - val_loss: 0.1292 - val_accuracy: 1.0000

Epoch 00003: val_loss improved from 0.19417 to 0.12916, saving model to /home/jupyter/train_0818/models/test_0818/tracking_model_seed1_tl9.h5
Epoch 4/10
24/24 [==============================] - 45s 2s/step - loss: 0.1009 - accuracy: 0.9459 - val_loss: 0.0888 - val_accuracy: 0.9688

Epoch 00004: val_loss improved from 0.12916 to 0.08879, saving model to /home/jupyter/train_0818/models/t

In [15]:
len(train_dict['y'])//batch_size

24

## Evaluate Model Performance

**Requires a Seed Value**

In [16]:
# Rosary added
## Redefine test_data

train_dict, test_dict = get_data(TEST_FILE, mode='sample', seed=train_seed, test_size=0.95)

In [17]:
test_data = datagen_test.flow(
    test_dict,
    batch_size=batch_size,
    seed=train_seed,
    crop_dim=crop_dim,
    neighborhood_scale_size=neighborhood_scale_size,
    min_track_length=min_track_length,
    features=features)

In [18]:
lst, y_true = next(test_data)
tracking_model.predict(lst)

array([[7.0123322e-04, 9.9929881e-01, 4.3245038e-10],
       [6.8697375e-03, 9.9312967e-01, 6.2089629e-07],
       [1.1312308e-03, 9.9886876e-01, 8.8368968e-10],
       [1.3791557e-01, 8.6208451e-01, 2.3772588e-09],
       [9.4766560e-04, 9.9905235e-01, 5.4560540e-11],
       [1.0024974e-02, 9.8997223e-01, 2.8130285e-06],
       [1.7404053e-03, 9.9825960e-01, 1.4009937e-09],
       [6.0133508e-04, 9.9939871e-01, 8.2486823e-11],
       [1.3870829e-02, 9.8612785e-01, 1.2442246e-06],
       [3.1541465e-04, 9.9968457e-01, 6.3187199e-11],
       [8.1088324e-04, 9.9918908e-01, 8.7988172e-11],
       [6.0824672e-04, 9.9939179e-01, 2.3000268e-10],
       [2.9492928e-04, 9.9970502e-01, 1.1275653e-11],
       [7.6412706e-04, 9.9923587e-01, 7.6230036e-11],
       [3.1828679e-04, 9.9968171e-01, 3.9050334e-12],
       [3.7461971e-03, 9.9625349e-01, 2.1083231e-07]], dtype=float32)

In [19]:
from sklearn.metrics import confusion_matrix

Y = []
Y_pred = []

for i in range(1,1000):
    if i % 100 == 0:
        print(".", end="")
    lst, y_true = next(test_data)
    y_true = np.argmax(y_true['classification'], axis=-1)
    y_pred = np.argmax(tracking_model.predict(lst), axis=-1)
    Y.append(y_true)
    Y_pred.append(y_pred)
    
Y = np.concatenate(Y, axis=0)
Y_pred = np.concatenate(Y_pred, axis=0)

print("")
cm = confusion_matrix(Y, Y_pred)
print(cm)

.........
[[3844 1420]
 [   0 9095]]


In [20]:
test_acc = sum(np.array(Y) == np.array(Y_pred)) / len(Y)
print('Accuracy across all three classes: ', test_acc)

# Normalize the diagonal entries of the confusion matrix
cm = cm.astype('float')/cm.sum(axis=1)[:, np.newaxis]
# Diagonal entries are the accuracies of each class
print('Accuracy for each individual class [Different, Same, Daughter]: ', cm.diagonal())

Accuracy across all three classes:  0.9011073194512152
Accuracy for each individual class [Different, Same, Daughter]:  [0.73024316 1.        ]
